In [1]:
import json
import w3lib.html
import random
import os
from datetime import datetime
import requests
import string
import calendar
import html
import locale 
from requests_html import HTMLSession
from requests_html import HTML
import validators
import pandas as pd
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
URL_SEED = 'https://www.elperiodista.cl/category/nacional/page/1/'
XPATH_HREF = "//h2/a/@href"

In [3]:
class Crawler():
    
    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]

    #Ruta XPATH hacia los enlaces que queremos recopilar
    xpath_href=XPATH_HREF

    #variable que permite listar todos los enlaces recopilado
    url_list = []
    
    def _init_(self, URL_SEED, XPATH_HREF):        
        pass

    
    def get(self,url):
        session = HTMLSession()
        header = dict()
        header['user-agent'] = random.choice(self.USER_AGENT_LIST)
        response = session.get(url,headers=header)        
        return response
    
    def parse(self, response):
        self.url_list = []
        urls = response.html.xpath(self.xpath_href)
        for url in urls:
            self.url_list.append(url)


In [4]:
urls=[]

for page in tqdm(range(3)): # revisemos sólo 2 páginas
    crawler = Crawler()
    response = crawler.get('https://www.elperiodista.cl/category/nacional/page/'+str(page))
    crawler.parse(response)

    
    for url in crawler.url_list:
        urls.append(url)

    crawler.url_list = urls
    
crawler.url_list

  0%|          | 0/3 [00:00<?, ?it/s]

['https://www.elperiodista.cl/2022/03/christian-little-asume-como-director-ejecutivo-de-conaf/',
 'https://www.elperiodista.cl/2022/03/investigacion-de-la-u-de-chile-advierte-peligro-de-tsunamis-en-la-zona-central/',
 'https://www.elperiodista.cl/2022/03/alcaldesa-hassler-aborda-desafios-en-seguridad-y-reactivacion-de-barrios-junto-a-ministra-del-interior-y-ministro-de-economia/',
 'https://www.elperiodista.cl/2022/03/diputados-udi-envian-carta-a-boric-cuestionando-nombramiento-de-barbara-figueroa/',
 'https://www.elperiodista.cl/2022/03/tribunal-constitucional-declara-inadmisible-recurso-de-brinks-y-reanuda-juicio-por-colusion-que-estaba-suspendido-ante-el-tdlc/',
 'https://www.elperiodista.cl/2022/03/comision-de-sistema-politico-acuerda-presidencialismo-atenuado-y-un-parlamentarismo-asimetrico-con-dos-camaras/',
 'https://www.elperiodista.cl/2022/03/senador-pugh-pide-urgente-trazabilidad-del-agua-que-usan-los-camiones-aljibes/',
 'https://www.elperiodista.cl/2022/03/partido-socialdem

In [7]:
len(urls)

42

In [8]:
urls[0]

'https://www.elperiodista.cl/2022/03/christian-little-asume-como-director-ejecutivo-de-conaf/'

In [9]:
URL_NEWS = urls[0]
XPATH_TITLE = "//h1"
XPATH_CONTENT ="//div[@class='continue-reading-content close']//p"

In [10]:
class Scraper:
    
    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]

    query_extract_title = XPATH_TITLE
    query_extract_text =  XPATH_CONTENT

    def _init_(self):
        pass
   
    def get(self,url):
        headers = {'user-agent':random.choice(self.USER_AGENT_LIST) }
        session = HTMLSession()
        response = session.get(url,headers=headers) 
        return response
    
    def clean_text(self, html_text):
        text_without_tags = w3lib.html.remove_tags(html_text)
        text_without_escape_chars = w3lib.html.replace_escape_chars(text_without_tags)
        text_without_escape_chars = html.unescape(text_without_escape_chars)
        text_without_whitespace = text_without_escape_chars.strip()
        return text_without_whitespace

    
    #### main function ####
    def parse(self, response):
        
        #URL
        self.news_url = response.url
        
        #TITLE
        self.news_title = response.html.xpath(self.query_extract_title)[0].text
        self.news_title = self.clean_text(self.news_title)
       
        
        #CONTENT
        self.news_text = ""
        
        html_paragraphs = response.html.xpath(self.query_extract_text)
        
        for paragraph in html_paragraphs:
            paragraph_content = w3lib.html.remove_tags(paragraph.text)    
            text_cleaned = self.clean_text(paragraph_content)
            self.news_text = self.news_text+text_cleaned


In [11]:
scraper = Scraper()
response =scraper.get(URL_NEWS)

In [12]:
scraper.parse(response)

In [13]:
scraper.news_title

'Christian Little asume como director ejecutivo de CONAF'

In [14]:
scraper.news_text

'Acompañado por el ministro (s) de Agricultura, José Guajardo, y con una visita a la Central Nacional de Coordinación de la Gerencia de Protección contra Incendios Forestales, Christian Little inició su gestión como director ejecutivo de la Corporación Nacional Forestal (CONAF), tras las reuniones protocolares sostenidas con la directora ejecutiva (i), Carmen Paz Medina, y el equipo gerencial de la institución, durante las primeras horas de la tarde del martes 29 de marzo.En sus primeras declaraciones al mando de la CONAF, el nuevo director de la institución indicó que entre sus retos destacan “mejorar la institucionalidad, que tiene que ver con la propuesta de un nuevo Servicio Nacional Forestal, que va de la mano con otro servicio, que es el Servicio de Biodiversidad y Áreas Protegidas. Es una agenda que tenemos como prioritaria. Otros desafíos dicen relación con aspectos legales asociados a la Ley de Bosque Nativo y reglamentos”.Añadió que también estará preocupado por el fortalecim

In [15]:
#Verificamos que el scraper devuelve algo
assert(len(scraper.news_title)>0)
assert(len(scraper.news_text)>0)

In [16]:
df = pd.DataFrame(data={'url': [URL_NEWS],'title':[scraper.news_title], 'content':[scraper.news_text]})
df

,url,title,content
0,https://www.elperiodista.cl/2022/03/christian-...,Christian Little asume como director ejecutivo...,"Acompañado por el ministro (s) de Agricultura,..."


In [17]:
XPATH_TITLE = "//h1"
XPATH_CONTENT ="//div[@class='continue-reading-content close']//p"

for url in tqdm(urls):
    URL_NEWS = url
    scraper = Scraper()
    response =scraper.get(URL_NEWS)
    scraper.parse(response)
    df1 = pd.DataFrame(data={'url': url, 'title':scraper.news_title, 'content':scraper.news_text},index=['url', 'title', 'content'])
    df = df.append(df1, ignore_index=True)
df = df.drop_duplicates()


  0%|          | 0/42 [00:00<?, ?it/s]

In [18]:
df

,url,title,content
0,https://www.elperiodista.cl/2022/03/christian-...,Christian Little asume como director ejecutivo...,"Acompañado por el ministro (s) de Agricultura,..."
4,https://www.elperiodista.cl/2022/03/investigac...,Investigación de la U. de Chile advierte pelig...,“Ruptura Compleja del Terremoto Illapel 2015 M...
7,https://www.elperiodista.cl/2022/03/alcaldesa-...,Alcaldesa Hassler aborda desafíos en seguridad...,"Esta mañana la alcaldesa de Santiago, Irací Ha..."
10,https://www.elperiodista.cl/2022/03/diputados-...,Diputados UDI envían carta a Boric cuestionand...,"El jefe de la Bancada UDI, Jorge Alessandri, j..."
13,https://www.elperiodista.cl/2022/03/tribunal-c...,Tribunal Constitucional declara inadmisible re...,La Primera Sala del Tribunal Constitucional (T...
16,https://www.elperiodista.cl/2022/03/comision-d...,Comisión de Sistema Político acuerda “presiden...,Constituyentes de todos los sectores llegaron ...
19,https://www.elperiodista.cl/2022/03/senador-pu...,Senador Pugh pide urgente Trazabilidad del Agu...,Tras conocer un reportaje denuncia que reveló ...
22,https://www.elperiodista.cl/2022/03/partido-so...,Partido Socialdemócrata de Chile hace su lanza...,Este sábado se desarrolló en la Plaza Ñuñoa de...
25,https://www.elperiodista.cl/2022/03/hora-del-p...,Hora del Planeta: tras apagar las luces el des...,La invitación a reflexionar y comprometerse co...
28,https://www.elperiodista.cl/2022/03/apunalan-a...,Apuñalan a joven trans y queda en riesgo vital,La mujer fue atacada con un arma cortante por ...
